# Test classification from model
## Load Packages

In [7]:
import torch
import torch.nn as nn
from torchvision.transforms import transforms
import numpy as np
from torch.autograd import Variable
from torchvision.models import squeezenet1_1
import torch.functional as F
from io import open
import os
from PIL import Image
import pathlib
import glob
import cv2

## set paths

In [10]:
train_path='scene_detection/seg_train/seg_train'
pred_path='scene_detection/seg_pred/seg_pred'

In [11]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

## CNN Network

In [12]:
#CNN Network


class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

In [13]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=6)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=6, bias=True)
)

## Transforms

In [14]:

transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

## Prediction function

In [15]:

def prediction(img_path,transformer):
    
    image=Image.open(img_path)
    
    image_tensor=transformer(image).float()
    
    
    image_tensor=image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor.cuda()
        
    input=Variable(image_tensor)
    
    
    output=model(input)
    
    index=output.data.numpy().argmax()
    
    pred=classes[index]
    
    return pred

In [16]:
images_path=glob.glob(pred_path+'/*.jpg')

In [17]:
pred_dict={}

for i in images_path:
    pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)

In [18]:
pred_dict

'sea',
 '20143.jpg': 'sea',
 '13589.jpg': 'street',
 '15846.jpg': 'sea',
 '14580.jpg': 'buildings',
 '16397.jpg': 'sea',
 '20625.jpg': 'buildings',
 '1780.jpg': 'sea',
 '21275.jpg': 'street',
 '6951.jpg': 'glacier',
 '7497.jpg': 'glacier',
 '12867.jpg': 'glacier',
 '572.jpg': 'sea',
 '7483.jpg': 'forest',
 '8968.jpg': 'sea',
 '21261.jpg': 'street',
 '957.jpg': 'forest',
 '13984.jpg': 'street',
 '23847.jpg': 'sea',
 '6574.jpg': 'sea',
 '18609.jpg': 'buildings',
 '20396.jpg': 'buildings',
 '4363.jpg': 'street',
 '21088.jpg': 'forest',
 '4405.jpg': 'street',
 '19271.jpg': 'forest',
 '2074.jpg': 'sea',
 '14755.jpg': 'street',
 '9121.jpg': 'sea',
 '6206.jpg': 'sea',
 '4411.jpg': 'forest',
 '2060.jpg': 'forest',
 '21922.jpg': 'glacier',
 '10241.jpg': 'forest',
 '3418.jpg': 'mountain',
 '19503.jpg': 'glacier',
 '16630.jpg': 'mountain',
 '13990.jpg': 'forest',
 '18635.jpg': 'street',
 '10269.jpg': 'mountain',
 '1227.jpg': 'sea',
 '12318.jpg': 'sea',
 '7130.jpg': 'sea',
 '4439.jpg': 'forest',
 